In [1]:
import time
import scipy
import numpy as np
import pandas as pd
import winsound

import networkx as nx
import matplotlib

from sklearn.preprocessing import LabelEncoder
import torch

from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import HGTConv, SAGEConv, GATConv, Linear, to_hetero
import torch_geometric.nn.functional as F

from HeteroDataFunctions import Encoder, add_types, complete_graph, flatten_lol, node_cat_dict, midi_type

print(scipy.__version__)
print(matplotlib.__version__)
print(nx.__version__)

1.7.3
3.6.2
2.8.4


In [2]:
# Complete Dataset
G = complete_graph(".\slac\embeddings\\all")

loading edgelists...
- notes.edgelist
- program.edgelist
- tempo.edgelist
- time.signature.edgelist
Nodes: 93553
Edges: 786635


In [3]:
nodes = pd.DataFrame((list(G.nodes)), columns=['name'])
edges = pd.DataFrame(np.array(list(G.edges)), columns=['source', 'target'])

In [4]:
node_categories = node_cat_dict(nodes)
node_categories.keys()

node_cat_dict took 0.38 secs to run


dict_keys(['note_group', 'pitch', 'program', 'MIDI', 'duration', 'velocity', 'time_sig', 'tempo'])

In [5]:
%%script false --no- raise -error

nodes_df_complete, edges_df_complete = add_types(nodes, edges, node_categories)

winsound.Beep(400, 700)

nodes_df_complete.to_csv('nodes_complete.csv')
edges_df_complete.to_csv('edges_complete.csv')

Couldn't find program: 'false'


In [6]:
nodes_df_complete = pd.read_csv('.\slac\Contents of Slac\\nodes_complete.csv')
edges_df_complete = pd.read_csv('.\slac\Contents of Slac\edges_complete.csv')
print('Done')

Done


In [7]:
edge_types = ["MIDI__has__tempo",
                   "MIDI__in__time_sig",
                   "MIDI__has__program",
                   "MIDI__has__note_group",
                   "note_group__has__velocity",
                   "note_group__has__duration",
                   "note_group__contains__pitch"]

In [8]:
full_categories = node_categories.copy()
full_categories['node_types'] = list(node_categories.keys())
full_categories['edge_types'] = edge_types  # Dictionary containing every string that may be found in our Dataframes
names_list_full = flatten_lol(full_categories.values())

len(names_list_full) == len(set(names_list_full))

True

In [9]:
encoder = Encoder(names_list_full, n_labels=5)

encoder.decode_value(5)

'g1601074'

In [10]:
node_types = set(nodes_df_complete['node_type'])
node_types

{'MIDI',
 'duration',
 'note_group',
 'pitch',
 'program',
 'tempo',
 'time_sig',
 'velocity'}

In [11]:
input_node_dict = {node_type: {'x': encoder.
                    encode_nodes(nodes_df_complete.
                    loc[nodes_df_complete['node_type'] == node_type, ['name']])}
                    for node_type in node_types}

encode_nodes took 0.04 secs to run
encode_nodes took 0.00 secs to run
encode_nodes took 5.91 secs to run
encode_nodes took 0.01 secs to run
encode_nodes took 0.00 secs to run
encode_nodes took 0.01 secs to run
encode_nodes took 0.04 secs to run
encode_nodes took 0.00 secs to run


In [12]:
input_node_dict['note_group']

{'x': tensor([[5.0000e+00],
         [6.0000e+00],
         [7.0000e+00],
         ...,
         [9.2486e+04],
         [9.2487e+04],
         [9.2488e+04]])}

In [13]:
edge_types

['MIDI__has__tempo',
 'MIDI__in__time_sig',
 'MIDI__has__program',
 'MIDI__has__note_group',
 'note_group__has__velocity',
 'note_group__has__duration',
 'note_group__contains__pitch']

In [14]:
edges_df_complete.loc[edges_df_complete['edge_type'] == 'MIDI__has__tempo', ['source', 'target']]

,source,target
936,Blues_-_Modern-Albert_King_-_Born_Under_A_Bad_...,11
571698,Blues_-_Modern-B_B_King_-_How_Blue_Can_You_Get,6
658833,Blues_-_Modern-B_B_King_-_Rock_Me_Baby,9
662826,Blues_-_Modern-B_B_King_-_The_Thrill_Is_Gone,9
666395,Blues_-_Modern-Buddy_Guy_-_Don't_Answer_the_Door,5
...,...,...
786125,Rock_-_Metal-Rage_Against_the_Machine_-_Bulls_...,8
786237,Rock_-_Metal-Rage_Against_the_Machine_-_Gueril...,11
786347,Rock_-_Metal-Rage_Against_the_Machine_-_Killin...,12
786441,Rock_-_Metal-Rage_Against_the_Machine_-_Know_Y...,8


In [15]:
input_edge_dict = {edge_type: {'edge_index': encoder.encode_edges(edges_df_complete.loc[
                    edges_df_complete['edge_type'] == edge_type, ['source', 'target']])} for edge_type in edge_types}

encode_edges took 0.05 secs to run
encode_edges took 0.03 secs to run
encode_edges took 0.19 secs to run
encode_edges took 16.18 secs to run
encode_edges took 13.62 secs to run
encode_edges took 21.88 secs to run
encode_edges took 1.54 mins to run


In [16]:
for key in input_edge_dict.keys():
    dim1 = input_edge_dict[key]['edge_index'].shape[0]
    input_edge_dict[key]['edge_index'] = input_edge_dict[key]['edge_index'].reshape(-1, dim1).to(torch.int32)

In [17]:
input_edge_dict

{'MIDI__has__tempo': {'edge_index': tensor([[92925, 93538, 92700, 93546, 92895, 93555, 92808, 93555, 92849, 93542,
           92835, 93539, 92864, 93555, 92924, 93555, 92789, 93542, 92784, 93542,
           92696, 93545, 92717, 93546, 92702, 93547, 92907, 93548, 92745, 93544,
           92929, 93547, 92749, 93541, 92911, 93541, 92891, 93555, 92764, 93538,
           92937, 93541, 92707, 93557, 92845, 93545, 92822, 93549, 92748, 93547,
           92741, 93547, 92811, 93549, 92704, 93555, 92714, 93547, 92921, 93554,
           92860, 93554, 92779, 93554, 92719, 93556, 92752, 93544, 92825, 93547,
           92880, 93556, 92754, 93547, 92730, 93554, 92788, 93555, 92803, 93555,
           92746, 93549, 92791, 93535, 92902, 93554, 92751, 93547, 92801, 93554,
           92800, 93538, 92733, 93547, 92728, 93554, 92900, 93554, 92903, 93554,
           92768, 93554, 92739, 93548, 92854, 93548, 92699, 93554, 92832, 93548,
           92931, 93554, 92798, 93544, 92742, 93544, 92775, 93535, 92701, 9

In [18]:
# Extract the label of each Midi.
midi_val = nodes_df_complete.loc[nodes_df_complete['node_type'] == 'MIDI', ['name']].values
midi_class = [midi_type(s[0]) for s in midi_val]

lb = LabelEncoder()
y = torch.from_numpy(lb.fit_transform(midi_class)).to(torch.float32)

lb.classes_

array(['Blues', 'Classical', 'Jazz', 'Rap', 'Rock'], dtype='<U9')

In [19]:
input_node_dict['MIDI']['y'] = y

In [33]:
H = HeteroData(input_node_dict, **input_edge_dict)

In [34]:
print(H)

HeteroData(
  MIDI={
    x=[250, 1],
    y=[250]
  },
  tempo={ x=[23, 1] },
  note_group={ x=[92484, 1] },
  program={ x=[108, 1] },
  velocity={ x=[11, 1] },
  pitch={ x=[93, 1] },
  duration={ x=[570, 1] },
  time_sig={ x=[14, 1] },
  (MIDI, has, tempo)={ edge_index=[2, 250] },
  (MIDI, in, time_sig)={ edge_index=[2, 239] },
  (MIDI, has, program)={ edge_index=[2, 1392] },
  (MIDI, has, note_group)={ edge_index=[2, 135160] },
  (note_group, has, velocity)={ edge_index=[2, 118626] },
  (note_group, has, duration)={ edge_index=[2, 92484] },
  (note_group, contains, pitch)={ edge_index=[2, 438484] }
)


In [35]:
H = T.ToUndirected()(H)

In [36]:
print(H)

HeteroData(
  MIDI={
    x=[250, 1],
    y=[250]
  },
  tempo={ x=[23, 1] },
  note_group={ x=[92484, 1] },
  program={ x=[108, 1] },
  velocity={ x=[11, 1] },
  pitch={ x=[93, 1] },
  duration={ x=[570, 1] },
  time_sig={ x=[14, 1] },
  (MIDI, has, tempo)={ edge_index=[2, 250] },
  (MIDI, in, time_sig)={ edge_index=[2, 239] },
  (MIDI, has, program)={ edge_index=[2, 1392] },
  (MIDI, has, note_group)={ edge_index=[2, 135160] },
  (note_group, has, velocity)={ edge_index=[2, 118626] },
  (note_group, has, duration)={ edge_index=[2, 92484] },
  (note_group, contains, pitch)={ edge_index=[2, 438484] },
  (tempo, rev_has, MIDI)={ edge_index=[2, 250] },
  (time_sig, rev_in, MIDI)={ edge_index=[2, 239] },
  (program, rev_has, MIDI)={ edge_index=[2, 1392] },
  (note_group, rev_has, MIDI)={ edge_index=[2, 135160] },
  (velocity, rev_has, note_group)={ edge_index=[2, 118626] },
  (duration, rev_has, note_group)={ edge_index=[2, 92484] },
  (pitch, rev_contains, note_group)={ edge_index=[2, 438

# GNN

In [39]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=64, out_channels=len(set(lb.classes_)))
model = to_hetero(model, H.metadata(), aggr='sum')

In [37]:
class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in H.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, H.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.lin_dict[node_type](x).relu_()

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)


        # return self.lin(x_dict['note_group'])


model = HGT(hidden_channels=64, out_channels=len(set(lb.classes_)),
            num_heads=2, num_layers=2)

In [38]:
with torch.no_grad():  # Initialize lazy modules.
    out = model(H.x_dict, H.edge_index_dict)

ValueError: Found indices in 'edge_index' that are larger than 22 (got 93557). Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 23) in your node feature matrix and try again.

In [27]:
print(H.x_dict)


{'MIDI': tensor([[92925.],
        [92700.],
        [92895.],
        [92808.],
        [92849.],
        [92835.],
        [92864.],
        [92924.],
        [92789.],
        [92784.],
        [92696.],
        [92717.],
        [92702.],
        [92907.],
        [92745.],
        [92929.],
        [92749.],
        [92911.],
        [92891.],
        [92764.],
        [92937.],
        [92707.],
        [92845.],
        [92822.],
        [92748.],
        [92741.],
        [92811.],
        [92704.],
        [92714.],
        [92921.],
        [92860.],
        [92779.],
        [92719.],
        [92752.],
        [92825.],
        [92880.],
        [92754.],
        [92730.],
        [92788.],
        [92803.],
        [92746.],
        [92791.],
        [92902.],
        [92751.],
        [92801.],
        [92800.],
        [92733.],
        [92728.],
        [92900.],
        [92903.],
        [92768.],
        [92739.],
        [92854.],
        [92699.],
        [92832.],
 

In [28]:
print(H.edge_index_dict)

{('MIDI', 'has', 'tempo'): tensor([[92925, 93538, 92700, 93546, 92895, 93555, 92808, 93555, 92849, 93542,
         92835, 93539, 92864, 93555, 92924, 93555, 92789, 93542, 92784, 93542,
         92696, 93545, 92717, 93546, 92702, 93547, 92907, 93548, 92745, 93544,
         92929, 93547, 92749, 93541, 92911, 93541, 92891, 93555, 92764, 93538,
         92937, 93541, 92707, 93557, 92845, 93545, 92822, 93549, 92748, 93547,
         92741, 93547, 92811, 93549, 92704, 93555, 92714, 93547, 92921, 93554,
         92860, 93554, 92779, 93554, 92719, 93556, 92752, 93544, 92825, 93547,
         92880, 93556, 92754, 93547, 92730, 93554, 92788, 93555, 92803, 93555,
         92746, 93549, 92791, 93535, 92902, 93554, 92751, 93547, 92801, 93554,
         92800, 93538, 92733, 93547, 92728, 93554, 92900, 93554, 92903, 93554,
         92768, 93554, 92739, 93548, 92854, 93548, 92699, 93554, 92832, 93548,
         92931, 93554, 92798, 93544, 92742, 93544, 92775, 93535, 92701, 93538,
         92783, 93538, 92

In [40]:
optimizer_name = "Adam"
lr = 1e-1
optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)


In [41]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(H.x_dict, H.edge_index_dict)
    mask = H['MIDI'].train_mask
    loss = F.cross_entropy(out['MIDI'][mask], H['MIDI'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)

In [42]:
train()

ValueError: Found indices in 'edge_index' that are larger than 249 (got 93557). Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 250) in your node feature matrix and try again.

 # Old Implementation

In [ ]:
# nodes_ten_ = encoder.encode_nodes(nodes_df_complete)
# edges_ten_ = encoder.encode_edges(edges_df_complete)

# node_type_ = nodes_df_complete.iloc[:, 1]

# Get the source and target indices from the edges tensor
# edge_index = edges_ten_[:, :2]

## Get the edge types from the edges tensor
#edge_type_ = edges_df_complete.iloc[:, 2]

#full_hetero_graph = HeteroData(x=nodes_ten_, node_type=node_type_, edge_index=edge_index, edge_type=edge_type_)